# Chapter 7 — Multi-Agent: Parallel

Agents run **concurrently** and their results are merged using `ParallelAgent`.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

In [ ]:
from google.adk.agents import Agent, ParallelAgent, SequentialAgent

# Three analysts that run in parallel
financial_analyst = Agent(
    name="financial_analyst",
    model="gemini-2.0-flash",
    description="Analyzes financial aspects.",
    instruction="Analyze the financial implications, costs, ROI, and economic impact of the given topic. Be concise.",
    output_key="financial_analysis",
)

risk_analyst = Agent(
    name="risk_analyst",
    model="gemini-2.0-flash",
    description="Analyzes risks and challenges.",
    instruction="Identify potential risks, challenges, and mitigation strategies for the given topic. Be concise.",
    output_key="risk_analysis",
)

opportunity_analyst = Agent(
    name="opportunity_analyst",
    model="gemini-2.0-flash",
    description="Identifies opportunities and advantages.",
    instruction="Identify opportunities, competitive advantages, and growth potential for the given topic. Be concise.",
    output_key="opportunity_analysis",
)

# Run all three in parallel
parallel_analysis = ParallelAgent(
    name="parallel_analysis",
    sub_agents=[financial_analyst, risk_analyst, opportunity_analyst],
)

# Then synthesize
synthesizer = Agent(
    name="synthesizer",
    model="gemini-2.0-flash",
    description="Combines all analyses into a final recommendation.",
    instruction="""Combine the analyses from session state:
- 'financial_analysis'
- 'risk_analysis'
- 'opportunity_analysis'

Produce a unified executive summary with a clear recommendation.""",
)

pipeline = SequentialAgent(
    name="analysis_pipeline",
    sub_agents=[parallel_analysis, synthesizer],
)

print("Pipeline: [financial + risk + opportunity] in parallel → synthesizer")

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=pipeline, app_name="parallel_demo", session_service=session_service)

session = await session_service.create_session(app_name="parallel_demo", user_id="user1")

request = types.Content(
    role="user",
    parts=[types.Part(text="Analyze the opportunity of launching an AI-powered customer service platform for mid-size businesses.")]
)

async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=request):
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text[:300]}...")
        print("---")

## Key Takeaways

- **`ParallelAgent`**: Runs multiple agents simultaneously for speed
- Each agent stores output via `output_key` for downstream consumption
- Combine with `SequentialAgent` for fan-out/fan-in patterns